<a href="https://colab.research.google.com/github/jhengineer/DeepLearning/blob/main/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy() #1차원 배열. 판다스의.
fish_target = fish['Species'].to_numpy()

In [4]:
# split tain/test
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42) # defautl 25%

#normalization
from sklearn.preprocessing import StandardScaler

ss=StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)


In [5]:
from sklearn.neighbors import KNeighborsClassifier

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.classes_)


['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


In [6]:
print(kn.predict(test_scaled[:5]))

proba = kn.predict_proba(test_scaled[:5]) # 확률 출력 probability.
print(np.round(proba, decimals=4)) # 4쨰자리지 끊어서.

'''
출력 수probability를 보면 n_neighbors=3이니, 1/3 or 2/3 or 1이 된다.
한계지. 이런거 말고 정말 확율같은 확율로.. 풀려면 로지스틱 회귀를 해.
'''

['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]


'\n출력 수probability를 보면 n_neighbors=3이니, 1/3 or 2/3 or 1이 된다.\n한계지. 이런거 말고 정말 확율같은 확율로.. 풀려면 로지스틱 회귀를 해.\n'

In [7]:
#Logistic Regression. 분류야. Regression 은 특정 숫자를 찾는거지만 로지스틱만 예외야. 분류.
'''
z = a * 무게 + b*길이 + c*대각선 + d* 높이 + e* 두께 + f

sigmod(z) = 1/(1+exp(-z))
'''
'''
처음 예제는 쉽게 가자 둘중 하나 고르는 이진분류 예시다
'''

bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

print(lr.predict(train_bream_smelt[:5]))

print(lr.predict_proba(train_bream_smelt[:5]))

# 알파벳 순서대로 bream이 음성 클래서 smelt 가 양성 클래스다.
# 아래 앞에있는 숫자 확률이 음성, 뒤에가 양성 이다.

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']
[[0.99760007 0.00239993]
 [0.02737325 0.97262675]
 [0.99486386 0.00513614]
 [0.98585047 0.01414953]
 [0.99767419 0.00232581]]


In [8]:
print(lr.coef_, lr.intercept_)

# 특성 해당하는 coeff 5개 절편 1개
# -0.404x무게-0.575x길이-0.66대각선 높이.. 두께.. -2.161(절)....@@@
# 표준화 노말라이제이션을 했기때문에.. ss 음수값들로만 생성되었다. 놀라지 말라. 맞다.

[[-0.40451732 -0.57582787 -0.66248158 -1.01329614 -0.73123131]] [-2.16172774]


In [9]:
decisions = lr.decision_function(train_bream_smelt[:5]) #선형방정식 자체값 위에  ....@@@ 식에 넣어서 얻어진값.
print(decisions)

[-6.02991358  3.57043428 -5.26630496 -4.24382314 -6.06135688]


In [10]:
from scipy.special import expit # scipy 안에 expit 함수가 시그모이드인데 요 함수를 써서 decisions 의 sigmoid 값을 알아보려고.
print(expit(decisions))
'''
시그모이드 값 출력된다. 사이파이에 exit 함수가 시그모이드다.)
0.5 넘으면 양성. 아래면 음성.
'''

[0.00239993 0.97262675 0.00513614 0.01414953 0.00232581]


'\n시그모이드 값 출력된다. 사이파이에 exit 함수가 시그모이드다.)\n0.5 넘으면 양성. 아래면 음성.\n'

In [11]:
#로지스틱으로 다중분류도 쉽게 할수 있다.

lr = LogisticRegression(C=20, max_iter=1000) # C는 규제다 default is L2 값이 크면 완와/ 작으면 강하다 1/c*람다.라서 크면 작고 작으면 크다.
# 소실함수 크로스엔트로피 뒤에 L2 L1 elastic .. 등등 option 이라 사용자 선택해서 써. 암튼 L2가 가장 안정적? 이라 해서 쓴다나..
# max_iter=1000 는 트레이닝을 1000번만 하라는거다. 모자르면 더 늘려.

lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))



0.9327731092436975
0.925
[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]
 [0.011 0.034 0.305 0.006 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [13]:
print(lr.coef_.shape, lr.intercept_.shape)

'''
(7,5) 해석 5개의 특성을 사용하는 7개의 선형방정. 절편 7개
각각 선형방정식을 하나씩 훈련해서 2진 분류를 7번 수행하는 방식과 비슷한 방식 <--역자 왈. 이해 잘 안됨..
나 왈 공부해 보니까...
one vs rest(OvR)로 선형 방정식 한개가 해당 클래스 vs 나머지 클래스로 나누는 모델이야.
기본적으로 사이키런의 로지스틱리그레션에서 제공한데. 멀티디지젼. 다중분류를 말이야.
이거 말고 (시그모이드 기반) softmax 하는것도 있어.
'''

(7, 5) (7,)


'\n(7,5) 해석 5개의 특성을 사용하는 7개의 선형방정. 절편 7개\n각각 선형방정식을 하나씩 훈련해서 2진 분류를 7번 수행하는 방식과 비슷한 방식 <--역자 왈. 이해 잘 안됨..\n나 왈 공부해 보니까...\none vs rest(OvR)로 선형 방정식 한개가 해당 클래스 vs 나머지 클래스로 나누는 모델이야. \n기본적으로 사이키런의 로지스틱리그레션에서 제공한데. 멀티디지젼. 다중분류를 말이야.\n이거 말고 (시그모이드 기반) softmax 하는것도 있어.\n'

In [16]:
decisions = lr.decision_function(test_scaled[:5]) # 위에서 구한 모델방정식에 넣은 값이다.
print(np.round(decisions, decimals=2))


#수동으로 사이파이 에서 제공하는 값. 공부해라.
from scipy.special import softmax
proba = softmax(decisions, axis=1)
print(np.round(proba, decimals=3))

[[ -6.51   1.04   5.17  -2.76   3.34   0.35  -0.63]
 [-10.88   1.94   4.78  -2.42   2.99   7.84  -4.25]
 [ -4.34  -6.24   3.17   6.48   2.36   2.43  -3.87]
 [ -0.69   0.45   2.64  -1.21   3.26  -5.7    1.26]
 [ -6.4   -1.99   5.82  -0.13   3.5   -0.09  -0.7 ]]
[[0.    0.014 0.842 0.    0.135 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.934 0.015 0.016 0.   ]
 [0.011 0.034 0.305 0.006 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
